<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1_Basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class RNN(nn.Module):
    def __init__(self,max_laps,hidden_size):
        super(RNN, self).__init__()
        self.embedding =  nn.Embedding(max_laps, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,max_laps)
        self.tanh = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

# Loss function (Based upon wk11 practical)

* Vocab size is not relevent here - going to give it binary classification values anyway.
* Everything else can be applied for time series: use Cross Entropy Loss, Length of the race = seq_len.
* For each lap, pass through the RNN with previous hidden state and build up hte matrix of outputs.
* Detact the previous hidden state as a new one will be generated.

In [ ]:
def calculate_loss(model,inputs,targets,hprev,max_tyre_life):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  #outputs = torch.zeros(0, max_laps)
  #print(outputs.shape)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(torch.tensor([inputs[t]]),hprev)
    # Gradually build up matrix of output logits of size seq_length * max_tyre_life
    #outputs = torch.cat((outputs ,y))

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(y,targets.unsqueeze(0))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

# My dataset - how to format?

* Training and testing
* inputs_training/ inputs_testing/  outputs_testing/ output_training
* For now, it will all just be a 2d array.

ERROR:

```2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552
   2553

IndexError: index out of range in self```

In [ ]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [ ]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            """if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])"""

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32)
            # Add to training data
            training_inputs.append(tyre_life_array)
            training_outputs.append(stint_change_array)

In [ ]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32)

            # Add to testing data
            testing_inputs.append(tyre_life_array)
            testing_outputs.append(stint_change_array)

# Training the model

In [ ]:
hidden_size = 1
lr = 0.01
iterations = 2000
max_race_size = int(max_race_size)

In [ ]:
def pad_with_last_value(tensor, max_race_size):
    if len(tensor) > max_race_size:
        return

    last_value = tensor[-1]
    pad_size = max_race_size - len(tensor)
    padding = torch.full((pad_size,), last_value, dtype=tensor.dtype)
    return torch.cat((tensor, padding))

In [ ]:
model = RNN(max_race_size, hidden_size)

optim = torch.optim.Adam(model.parameters(), lr = lr)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size,) # reset RNN memory

  inputs = training_inputs[p]
  targets = training_outputs[p]

  inputs = pad_with_last_value(inputs,max_race_size)
  targets = pad_with_last_value(targets,max_race_size)


  # Compute loss for current subsequence
  loss, hprev = calculate_loss(model,inputs,targets,hprev,max_race_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))

  n += 1 # iteration counter
  p += 1 # dataset counter

  #Restart to the front of the list.
  if p == len(training_inputs) -1:
    p = 0

[0] Loss: 0.73
[500] Loss: 0.70
[1000] Loss: 0.73
[1500] Loss: 0.30
[2000] Loss: 0.52


#How can we calculate accuracy for this basic model?

* Say is probability is above 80% percent pit, otherwise don't pit.
* Want at least one right pit stop identification to class it as match.

In [ ]:
for t in range(len(testing_inputs[1])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor([testing_inputs[1][t]]),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities

tensor([[1.0000e+00, 8.6562e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9097e-01,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9995e-01, 1.0000e+00, 1.0000e+00,
         4.6142e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9996e-01,
         1.0000e+00, 1.0000e+00, 1.2703e-04, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 8.2599e-01, 1.0000e+00, 9.7841e-01, 8.7177e-01,
         1.0000e+00, 9.4194e-01, 1.0000e+00, 8.0562e-01, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 7.8653e-08, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 9.9970e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e

In [ ]:
for t in range(len(training_inputs[0])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor([training_inputs[0][t]]),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities

tensor([[9.6322e-01, 6.7875e-03, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9910e-01,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 4.3098e-03, 9.9997e-01, 1.0000e+00,
         9.7848e-01, 9.9999e-01, 9.9996e-01, 1.0000e+00, 9.9531e-01, 9.6623e-01,
         1.0000e+00, 1.0000e+00, 7.6865e-05, 9.9287e-01, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 9.8730e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         9.9860e-01, 1.9398e-08, 1.5346e-03, 1.0000e+00, 9.9837e-01, 2.8660e-01,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 9.2517e-01, 9.8466e-01, 8.7852e-01,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9989e-01, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 5.5465e-02, 1.0000e+00, 1.0000e+00, 9.9999e-01,
         1.0000e+00, 1.0000e+00, 9.9975e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e

In [225]:
import torch

def evaluate_model(inputs, outputs, tolerance=3):
    total_correct = 0
    total_predictions = 0

    for seq_idx in range(len(inputs)):
        # Initialize hidden state for each sequence
        hprev = torch.zeros(hidden_size,)

        # Get the input and target sequences
        input = inputs[seq_idx]
        output = outputs[seq_idx].numpy()

        # Step through the input sequence and get predicted probabilities
        for t in range(len(input)):
            hprev, y = model(torch.tensor([input[t]]), hprev)
            predicted_pits = torch.sigmoid(y) # Apply sigmoid to logits to get probabilities

        predicted_pits = predicted_pits.detach().numpy()[0]
        predicted_pits = predicted_pits > 0.5
        actual_pits = output == 1.0

        # Now check accuracy for this sequence with tolerance
        correct_count = 0
        total_predictions_for_sequence = 0

        for predicted_pit_idx in range(len(predicted_pits)):
          if predicted_pits[predicted_pit_idx]:  # If a pit stop is predicted
              total_predictions += 1

              # Calculate the tolerance window around the predicted pit
              start_idx = max(0, predicted_pit_idx - tolerance)
              end_idx = min(len(actual_pits), predicted_pit_idx + tolerance + 1)

              # Check if there's any actual pit stop within this window using NumPy
              if np.any(actual_pits[start_idx:end_idx] == 1):  # If there's a pit stop in this window
                  correct_count += 1
                  break

        total_correct += correct_count

    # Calculate overall accuracy
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    return accuracy * 100

# Plot to see what this basic model has remembered - probably just averages

# Validating my data

In [ ]:
len(training_inputs) == len(training_outputs)

True

In [ ]:
training_inputs[0].shape

torch.Size([54])

In [ ]:
training_outputs[0].shape

torch.Size([54])